In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
mnist = tf.keras.datasets.mnist

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9999):
      print("\nReached 99% accuracy so cancelling training")
      self.model.stop_training = True

In [5]:
callback = myCallback()

In [6]:
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(2, 2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPool2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
])

In [7]:
model.compile(optimizer='rmsprop',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [8]:
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), callbacks=callback)

Epoch 1/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.1265 - accuracy: 0.9617 - val_loss: 0.0579 - val_accuracy: 0.9823
Epoch 2/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0454 - accuracy: 0.9868 - val_loss: 0.0390 - val_accuracy: 0.9865
Epoch 3/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0334 - accuracy: 0.9897 - val_loss: 0.0341 - val_accuracy: 0.9882
Epoch 4/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0272 - accuracy: 0.9923 - val_loss: 0.0310 - val_accuracy: 0.9895
Epoch 5/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0221 - accuracy: 0.9935 - val_loss: 0.0400 - val_accuracy: 0.9887
Epoch 6/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0197 - accuracy: 0.9944 - val_loss: 0.0318 - val_accuracy: 0.9909
Epoch 7/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0168 - accuracy: 0.9952 - val_loss: 0.0430 - val_ac

In [9]:
test = pd.read_csv('test.csv')
test = test / 255.
test = test.values.reshape(-1, 28, 28, 1)

In [10]:
label = model.predict(test)

In [11]:
label = np.squeeze(label)

In [12]:
label = tf.argmax(label, axis=1)

In [13]:
results = pd.Series(label, name='Label')

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"), results],axis = 1)

In [14]:
submission.tail(5)

,ImageId,Label
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
27999,28000,2


In [22]:
submission.to_csv("digit_submission_9.csv",index=False)